In [4]:
%cd D:\YeqinHUANG_Collection\ucl\week7_03.Dec.2023\Skillclass2 text

D:\YeqinHUANG_Collection\ucl\week7_03.Dec.2023\Skillclass2 text


In [265]:
import re
import ebooklib
from ebooklib import epub

def preprocess(epub_file, ID):
    def read_epub_book(epub_file):
        book = epub.read_epub(epub_file)
        text = ''

        for item in book.get_items():
            if item.get_type() == ebooklib.ITEM_DOCUMENT:
                content = item.get_content().decode('utf-8')
                content = re.sub('<[^<]+?','',content) #remove the html
                content = re.sub('\s+',' ',content) #remove the black
                content = re.sub('\n',' ',content)#remove the new line
                content = re.sub("&#13;", ' ', content)# remove the enter
                text += content
                return text

    def merge_strings_until_limit(strings, min_length, max_length, test_for_max=0):
        merged_string = ""
        merged_strings = []

        for s in strings:
            if len(merged_string) <= min_length:# and len(s)<max_length:
                merged_string += s

            elif len(merged_string) > max_length and test_for_max<5:
                splitParagraph = merged_string.split('.')
                splitParagraphRePoint = []
                for sp in splitParagraph:
                    splitParagraphRePoint.append(sp+'.')
                merged = merge_strings_until_limit(splitParagraphRePoint, min_length, max_length, test_for_max+1)
                merged_strings.extend(merged)
                merged_string = s
            else:
                merged_strings.append(merged_string)
                merged_string = s

        if merged_string:
            merged_strings.append(merged_string)

        return merged_strings

    def read_epub_paragraphs(epub_file, ID):
        book = epub.read_epub(epub_file)
        paragraphs = []

        for item in book.get_items():
            if item.get_type() == ebooklib.ITEM_DOCUMENT:
                content = item.get_content().decode('utf-8')
                content = re.sub('<[^<]+?>', '', content)  # Remove HTML tags
                content = re.sub('\s+', ' ', content)  # Replace multiple whitespaces with a single space
                content = re.sub('\n', ' ', content)
                content = content.strip()  # Remove leading and trailing spaces
                if len(content) > 50:
                    paragraph_dict = {'paragraph': content, 'bookID': ID}
                    paragraphs.append(paragraph_dict)
        return paragraphs

read_epub_paragraphs('rohmer-tales-of-chinatown.epub','rohmer-tales-of-chinatown')

[{'paragraph': 'Tales of Chinatown Sax Rohmer Copyright © 2014 epubBooks All Rights Reserved. This publication is protected by copyright. By payment of the required fees, you have been granted the non-exclusive, non-transferable right to access and read the text of this ebook on-screen or via personal text-to-speech computer systems. No part of this text may be reproduced, transmitted, downloaded, decompiled, reverse engineered, stored in or introduced into any information storage and retrieval system, in any form or by any means, whether electronic or mechanical, now known or hereinafter invented, without the express written permission of epubBooks. www.epubbooks.com',
  'nr': 1,
  'bookID': 'rohmer-tales-of-chinatown'},
 {'paragraph': 'The Daughter of Huang ChowI: "Diamond Fred" In the saloon bar of a public–house, situated only a few hundred yards from the official frontier of Chinatown, two men sat at a small table in a corner, engaged in earnest conversation.',
  'nr': 2,
  'bookI

In [266]:
processedDoc = [p['paragraph'] for p in read_epub_paragraphs('rohmer-tales-of-chinatown.epub', 'rohmer-tales-of-chinatown')]

In [267]:
processedDoc

['Tales of Chinatown Sax Rohmer Copyright © 2014 epubBooks All Rights Reserved. This publication is protected by copyright. By payment of the required fees, you have been granted the non-exclusive, non-transferable right to access and read the text of this ebook on-screen or via personal text-to-speech computer systems. No part of this text may be reproduced, transmitted, downloaded, decompiled, reverse engineered, stored in or introduced into any information storage and retrieval system, in any form or by any means, whether electronic or mechanical, now known or hereinafter invented, without the express written permission of epubBooks. www.epubbooks.com',
 'The Daughter of Huang ChowI: "Diamond Fred" In the saloon bar of a public–house, situated only a few hundred yards from the official frontier of Chinatown, two men sat at a small table in a corner, engaged in earnest conversation.',
 ' They afforded a sharp contrast. One was a thick–set and rather ruffianly looking fellow, not too 

In [268]:
# !pip install scikit-learn

In [269]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [270]:
vectorizer = TfidfVectorizer(min_df=2)

In [271]:
tfidf_matrix = vectorizer.fit_transform(processedDoc)

In [272]:
tfidf_matrix

<1624x4198 sparse matrix of type '<class 'numpy.float64'>'
	with 59488 stored elements in Compressed Sparse Row format>

In [273]:
len(processedDoc)

1624

In [274]:
vectorizer.get_feature_names_out()

array(['1912', 'able', 'aboard', ..., 'zahara', 'zani', 'zest'],
      dtype=object)

In [275]:
len(vectorizer.get_feature_names_out())

4198

In [276]:
from sklearn.metrics.pairwise import cosine_similarity

In [277]:
query = 'temples and the moon with a man'
processedQuery = vectorizer.transform([query])[0]
print(processedQuery)

  (0, 4125)	0.21838807041821348
  (0, 3730)	0.10129565699710373
  (0, 3711)	0.6641030647153039
  (0, 2375)	0.6429540740558068
  (0, 2258)	0.26883025598206894
  (0, 138)	0.12390848288931108


In [341]:
processedQuery = 'walk on the street with girlfriend'

In [342]:
query_vector = vectorizer.transform([processedQuery])

In [343]:
query_vector

<1x4198 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [344]:
similarities = cosine_similarity(query_vector, tfidf_matrix)

In [345]:
similarities

array([[0.03346246, 0.01627495, 0.00474753, ..., 0.04353697, 0.01038952,
        0.01704629]])

In [346]:
len(similarities[0])

1624

In [347]:
nearest_neighbor_index = similarities.argmax()
nearest_neighbor = tfidf_matrix[nearest_neighbor_index]
similarity_score = similarities[0, nearest_neighbor_index]

In [348]:
nearest_neighbor_index, similarity_score

(50, 0.24800057649167095)

In [349]:
paragraph_info = processedDoc[50]
print(paragraph_info)

 "Four convictions; one, robbery with violence." Jim Poland nodded sullenly. "You were arrested at the corner of Pekin Street about midnight. What were you doing there?" "Taking a walk." "I'll say it again," rapped Kerry, fixing his fierce eyes upon the man's face.


# Reduce dimensionality

In [333]:
import numpy as np
from scipy.sparse import random
from sklearn.decomposition import TruncatedSVD

In [334]:
n_components = 250  
svd = TruncatedSVD(n_components=n_components, algorithm='randomized')
reduced_matrix = svd.fit_transform(tfidf_matrix)

In [350]:
reduced_query_vec = svd.transform(query_vector)

In [351]:
len(reduced_query_vec[0])

250

In [352]:
similarities2 = cosine_similarity(reduced_query_vec, reduced_matrix)

In [353]:
nearest_neighbor_index = similarities2.argmax()
nearest_neighbor = tfidf_matrix[nearest_neighbor_index]
similarity_score = similarities2[0, nearest_neighbor_index]

In [354]:
nearest_neighbor_index

281

In [356]:
paragraphs[281]

' Opposite the corner of Bond Street he stood still, swinging his malacca cane and gazing fixedly along this narrow bazaar street of the Baghdad of the West. His trim, athletic figure was muffled in a big, double–breasted, woolly overcoat, the collar turned up about his ears.'

# Vectorise Text - 2

In [502]:
import gensim
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import nltk
from bs4 import BeautifulSoup
import ebooklib
from ebooklib import epub

In [545]:
def preprocess2(text):
    def clean_html_tags(text):
        if isinstance(text, str):
            soup = BeautifulSoup(text, 'html.parser')
            cleaned_text = soup.get_text()
            return cleaned_text
        else:
            return ''

    cleaned_text = clean_html_tags(text)

    words = gensim.utils.simple_preprocess(cleaned_text, min_len=2, deacc=True)

    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

    stop_words = set(stopwords.words("english"))
    english_words = set(nltk.corpus.words.words())

    def is_english_word(word):
        return word in english_words

    filtered_words = [word for word in lemmatized_words if word not in stop_words and is_english_word(word)]

    return filtered_words

epub_file = 'rohmer-tales-of-chinatown.epub'
text = extract_text_from_epub(epub_file)  
processed_text = preprocess2(text)
print(processed_text)

['tale', 'table', 'content', 'daughter', 'chow', 'diamond', 'end', 'secret', 'treasure', 'house', 'personal', 'report', 'detective', 'chief', 'inspector', 'kerry', 'officer', 'charge', 'limehouse', 'inquiry', 'hint', 'incense', 'sound', 'cage', 'bird', 'picture', 'pad', 'coffin', 'kerry', 'red', 'kerry', 'duty', 'jack', 'room', 'golden', 'dan', 'kerry', 'junior', 'knight', 'errant', 'retribution', 'pigtail', 'hi', 'wing', 'ho', 'lost', 'ended', 'house', 'golden', 'joss', 'blood', 'idol', 'lung', 'captain', 'dan', 'secret', 'man', 'shaven', 'skull', 'strange', 'disappearance', 'clue', 'photograph', 'house', 'river', 'harem', 'agency', 'white', 'hat', 'major', 'jack', 'curious', 'outrage', 'criminal', 'genius', 'slanting', 'eye', 'rose', 'black', 'mass', 'dance', 'veil', 'house', 'star', 'hand', 'mandarin', 'shadow', 'curtain', 'lady', 'gold', 'cased', 'nail', 'key', 'temple', 'heaven', 'keeper', 'key', 'tiger', 'lady', 'twin', 'pool', 'amber', 'living', 'ivory', 'god', 'madame', 'smile'

In [547]:
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

paragraphs = read_epub_paragraphs('rohmer-tales-of-chinatown.epub', 'rohmer-tales-of-chinatown')
documents = [TaggedDocument(words=preprocess2(p['paragraph']), tags=[i]) for i, p in enumerate(paragraphs)]

model = Doc2Vec(vector_size=250, min_count=5, alpha=0.025, min_alpha=0.025, epochs=100)

model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

In [548]:
print(model.wv.index_to_key)

['wa', 'said', 'one', 'upon', 'man', 'room', 'eye', 'kerry', 'could', 'hand', 'door', 'know', 'see', 'would', 'night', 'like', 'little', 'face', 'time', 'house', 'come', 'way', 'stood', 'old', 'moment', 'place', 'friend', 'woman', 'came', 'back', 'good', 'found', 'knew', 'inspector', 'never', 'voice', 'street', 'great', 'well', 'think', 'looking', 'two', 'made', 'chief', 'ha', 'girl', 'thing', 'must', 'long', 'something', 'table', 'even', 'tell', 'might', 'right', 'sir', 'light', 'much', 'u', 'presently', 'ah', 'yes', 'life', 'last', 'ever', 'first', 'seen', 'look', 'case', 'go', 'mean', 'fact', 'figure', 'sort', 'end', 'yet', 'limehouse', 'nothing', 'may', 'word', 'half', 'another', 'turned', 'although', 'many', 'almost', 'saw', 'lady', 'strange', 'name', 'boy', 'arm', 'say', 'window', 'chow', 'make', 'thought', 'left', 'shoulder', 'take', 'dark', 'got', 'head', 'foot', 'beside', 'quite', 'away', 'father', 'cried', 'lay', 'story', 'enough', 'without', 'side', 'open', 'madame', 'big', 

In [549]:
test = model.predict_output_word(['old', 'house', 'come', 'woman'])
print(test)

[('door', 0.37454766), ('lung', 0.10659277), ('mandarin', 0.100741625), ('treasure', 0.08203032), ('wise', 0.039402768), ('shoulder', 0.034565825), ('chow', 0.022951106), ('used', 0.015782287), ('round', 0.014320126), ('word', 0.012731582)]


In [550]:
query = 'a man thinking about his life under the full moon'
tokens = preprocess2(query)
vector = model.infer_vector(tokens)


In [552]:
vector

array([ 1.2129875 ,  0.8529779 , -0.20634152,  0.5259612 ,  0.5505545 ,
        0.47606218,  0.5506703 ,  0.8832013 , -0.15185846,  0.18276058,
       -0.08754498,  0.40813243,  1.1057224 ,  0.7009827 , -0.30069315,
       -0.64249575, -0.7618696 , -0.54641485, -0.18350469,  0.5938165 ,
       -0.7622046 ,  0.16734947, -0.06613606, -0.24302034,  0.68695337,
       -1.1543721 , -0.4543094 , -0.18569757, -0.37005395,  0.06015196,
        0.4827856 ,  0.05298766, -0.57499075,  0.5582063 ,  1.0249138 ,
       -0.1850381 , -0.87482613, -0.06291874, -0.18319897,  1.0650373 ,
        0.24838346,  0.32309392, -0.8502213 ,  0.39374015,  0.6649338 ,
       -0.49534988,  0.03754071,  0.02163574, -0.2113795 ,  0.82233584,
       -0.17356831, -0.11806457, -0.8296613 , -0.01432387, -0.40079957,
        0.38421756,  0.34342292, -0.26108515,  0.5415134 , -0.9176436 ,
       -0.6156699 ,  0.5196696 , -1.1319448 ,  0.8369223 ,  0.2431234 ,
        0.39773148, -0.13998155, -0.7651883 , -0.46496847, -0.66

In [553]:
model.dv.most_similar(positive=[vector])

[(142, 0.37442025542259216),
 (981, 0.37431037425994873),
 (1185, 0.3647216558456421),
 (498, 0.33954548835754395),
 (1254, 0.33749473094940186),
 (1153, 0.32967913150787354),
 (963, 0.3292272090911865),
 (388, 0.32649704813957214),
 (1023, 0.3062390089035034),
 (1500, 0.30227404832839966)]

In [554]:
paragraphs[142]

{'paragraph': ' There is a night life in Limehouse, as he had learned, but it is a mole life, a subterranean life, of which no sign appears above ground after a certain hour. Nevertheless, as he entered the area which harbours those strange, hidden resorts the rumour of which has served to create the glamour of Chinatown, he found himself to be thinking of the great influence said to be wielded by Huang Chow, and wondering if unseen spies watched his movements.',
 'nr': 146,
 'bookID': 'rohmer-tales-of-chinatown'}